In [129]:
import numpy as np
import sympy as sp

In [130]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return list(map(list, chain.from_iterable(combinations(s, r) for r in range(len(s)+1))))

In [131]:
def submat(mat, rs, cs):
    return mat[np.ix_(rs, cs)]

def minor(mat, rs, cs):
    return sp.Matrix(submat(mat, rs, cs)).det()
    
def initialMinor(mat, i, j):
    size = 0

    if i < j:
        size = i
    else:
        size = j
    
    rs = list(range(i - size, i + 1))
    cs = list(range(j - size, j + 1))

    return minor(mat, rs, cs)

def totallyPositive(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if initialMinor(mat, i, j) <= 0:
                return False
    return True

def totallyNonnegative(mat):
    n = mat.shape[0]
    combs = powerset(list(range(n)))
    combs = [(c1, c2) for c1 in combs for c2 in combs if len(c1) == len(c2)]
    
    for rs, cs in combs:
            if minor(mat, rs, cs) < 0:
                return False
    return True
        

In [133]:
class PlanarNet:
    def __init__(self, mat):
        self.n = mat.shape[0]
        self.diags = [0 for _ in range(2 * self.n)]
        for level in range(mat.shape[0]):
            self.diags[level] = np.flip(mat[level:, level])
            self.diags[-(level + 1)] = mat[level, level:]
    
    def fromDiags(diags):
        if len(diags) % 2 != 0:
            raise RuntimeError("Bad diags: len mod 2 != 0")
            
        net = PlanarNet(np.zeros((1, 1)))
        net.n = len(diags) // 2
        net.diags = diags
        return net
    
    def setEdge(self, level, idx, val):
        self.diags[level].setIdx(idx, val)
    
    def __repr__(self):
        return ((self.diags)).__repr__()
    
    def __eq__(self, other):
        for d1, d2 in zip(self.diags, other.diags):
            if len(d1) != len(d2):
                return False

            if not (d1 == d2).all():
                return False
        
        if other.n != self.n:
            return False
            
        return True

# Matrix -> Planar Net

In [134]:
def MatToNet(mat):
    n = mat.shape[0]
    M = np.zeros_like(mat)
    A = np.zeros_like(mat)
    P = np.zeros_like(mat)

    for level in range(n):
        M[level, level] = initialMinor(mat, level, level)
        prevMinor = 1 if level == 0 else M[level - 1, level - 1]
        A[level, level] = M[level, level] / prevMinor
        P[level, level] = A[level, level]

        for i in range(level + 1, n):
            M[level, i] = initialMinor(mat, level, i)
            prevMinor = 1 if level == 0 else M[level - 1, i - 1]
            A[level, i] = M[level, i] / (P[level, i - 1] * prevMinor)
            P[level, i] = A[level, i] * P[level, i - 1]

        for i in range(level + 1, n):
            M[i, level] = initialMinor(mat, i, level)
            prevMinor = 1 if level == 0 else M[i - 1, level - 1]
            A[i, level] = M[i, level] / (P[i - 1, level] * prevMinor)
            P[i, level] = A[i, level] * P[i - 1, level]

    return PlanarNet(A)

# Planar Net -> Matrix

In [135]:
class Graph:
    def __init__(self, graph_list):
        self.graph_list = graph_list
    
    def v(self, id):
        return self.graph_list[id]

    def __repr__(self):
        ans = ""
        for k in self.graph_list:
            ans += str(k)
            ans += ': '
            ans += str(self.graph_list[k])
            ans += '\n'
        return ans
    

### Planar Net -> Graph

In [136]:
def NetToGraph(net):
    n = net.n
    gr = {}

    for level in range(n):
        gr[(level, 0, 0)] = [(level, 0, 1, 1)]

        for v in range(1, level + 1):
            gr[(level, 0, v)] = []
            gr[(level, 0, v)].append((level, 0, v + 1, 1))
            if level > 0:
                gr[(level, 0, v)].append((level - 1, 0, v, net.diags[v - 1][n - (level + 1)]))
        
        gr[(level, 0, level + 1)] = [(level, 1, 0, net.diags[level][-1])]

        for v in range(level + 1):
            gr[(level, 1, v)] = []
            gr[(level, 1, v)].append((level, 1, v + 1, 1))
            if level < n - 1:
                gr[(level, 1, v)].append((level + 1, 1, v + 1, net.diags[-(level - v + 1)][v + 1]))
        
        gr[(level, 1, level + 1)] = []

    return Graph(gr)

### DFS

In [137]:
def dfs(gr, v, to, cumm=1):
    if v == to:
        return cumm
    
    ans = 0

    for n in gr.v(v):
        ans += dfs(gr, (n[0], n[1], n[2]), to, cumm * n[3])
    
    return ans

In [138]:
def NetToMat(net):
    gr = NetToGraph(net)
    n = net.n
    mat = np.ones((n, n))

    for i in range(0, n):
        for j in range(0, n):
            mat[i, j] = dfs(gr, (i, 0, 0), (j, 1, j + 1))
    
    return mat

# Test

## 3x3 matrix from article tests

In [139]:
def testMat(a, b, c, d, e, f, g, h, i):
    return np.array([
        [d, d * h, d * h * i],
        [b * d, b * d * h + e, b * d * h * i + e * g + e * i],
        [a * b * d, a * b * d * h + a * e + c * e, a * b * d * h * i + (a + c) * e * (g + i) + f]
    ])

def testMatToNet():
    x = np.random.randint(1, 10, size=(9))
    mat = testMat(*x)
    net = MatToNet(mat)

    if list(net.diags[0]) != [x[0], x[1], x[3]] or \
       list(net.diags[1]) != [x[2], x[4]] or \
       list(net.diags[2]) != [x[5]] or \
       list(net.diags[3]) != [x[5]] or \
       list(net.diags[4]) != [x[4], x[6]] or \
       list(net.diags[5]) != [x[3], x[7], x[8]]:
        print('Wrong answer')
        print(mat)


def testMatToNetToMat():
    x = np.random.randint(1, 10, size=(9))
    mat = testMat(*x)
    
    if not (NetToMat(MatToNet(mat)) == mat).all():
        print('Wrong answer')
        print(mat)

def testNetToMatToNet():
    for size in range(2, 5):
        diags = [0 for _ in range(2 * size)]
        for i in range(size):
            x = np.random.randint(1, 3, size=(2 * (size - i) - 1))
            diags[i] = x[:(size - i)]
            diags[-(i + 1)] = x[(size - i - 1):]
        
        net = PlanarNet.fromDiags(diags)
        if net != MatToNet(NetToMat(net)):
            print('Wrong answer')
            print(net)

In [140]:
for _ in range(10000):
    testMatToNet()

In [141]:
for _ in range(10000):
    testMatToNetToMat()

In [142]:
for _ in range(10000):
    testNetToMatToNet()

## Total positivity tests

In [143]:
for _ in range(1000):
    for size in range(2, 7):
        diags = [0 for _ in range(2 * size)]
        for i in range(size):
            x = np.random.randint(1, 10, size=(2 * (size - i) - 1))
            diags[i] = x[:(size - i)]
            diags[-(i + 1)] = x[(size - i - 1):]
        
        net = PlanarNet.fromDiags(diags)
        mat = NetToMat(net)
        if not totallyPositive(mat):
            print('Wrong answer', net, mat)

## Total non-negativity tests

In [144]:
for _ in range(100):
    for size in range(2, 7):
        diags = [0 for _ in range(2 * size)]
        for i in range(size):
            x = np.random.randint(0, 2, size=(2 * (size - i) - 1))
            diags[i] = x[:(size - i)]
            diags[-(i + 1)] = x[(size - i - 1):]
        
        net = PlanarNet.fromDiags(diags)
        mat = NetToMat(net)
        if not totallyNonnegative(mat):
            print('Wrong answer', net, mat)

## Tests for $e \in \mathbb{R}$

In [145]:
for _ in range(100):
    for size in range(2, 6):
        diags = [0 for _ in range(2 * size)]
        for i in range(size):
            x = np.random.random((2 * (size - i) - 1))
            diags[i] = x[:(size - i)]
            diags[-(i + 1)] = x[(size - i - 1):]
        
        net = PlanarNet.fromDiags(diags)
        mat = NetToMat(net)
        if not totallyNonnegative(mat):
            print('Wrong answer', net, mat)

In [146]:
for _ in range(1000):
    for size in range(2, 6):
        diags = [0 for _ in range(2 * size)]
        for i in range(size):
            x = np.random.random((2 * (size - i) - 1))
            x += 0.01 #to delete zeros
            diags[i] = x[:(size - i)]
            diags[-(i + 1)] = x[(size - i - 1):]
        
        net = PlanarNet.fromDiags(diags)
        mat = NetToMat(net)
        if not totallyPositive(mat):
            print('Wrong answer', net, mat)

# Big matrices

In [149]:
for _ in range(100):
    size = 10
    diags = [0 for _ in range(2 * size)]
    for i in range(size):
        x = np.random.randint(1, 3, size=(2 * (size - i) - 1))
        diags[i] = x[:(size - i)]
        diags[-(i + 1)] = x[(size - i - 1):]
    net = PlanarNet.fromDiags(diags)
    mat = NetToMat(net)
    if not totallyPositive(mat):
        print('Wrong answer', net, mat)